<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=af56f5aee3c45372dd2f79bb9f76fa048a0e3a2d029da356f65fd05a093e65d3
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-06 14:07:51
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.24 C
CY Investment:  1.51 C
Reserve:  57.44 K
Current:  1.41 C
-------------------
Today PnL: -13.86 K (-0.1%)
Current PnL: -20.65 L (-13.71%)
CY Booked + Current PnL: -9.21 L (-6.11%)
-------------------
Total profit:  1.38 L
Total loss:  -22.03 L
-------------------
Total Booked + Current PnL: 17.77 L (14.37%)
Total Booked PnL: 38.42 L (31.06%)
Curr Year Booked PnL: 11.44 L (8.12%)
Prev Year Booked PnL: 26.98 L (21.81%)
Est FTT:  2.29 C
Est FTT PnL: 87.85 L (62.34%)
Deployed:  1.24 C
Current:  1.41 C
CAGR/XIRR %: 7.93%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.39,115354.0,14850.0,14258.0,-1.59,14.78,12.36,28.96,79.0,1.04,0.82,53.76,MH,ATH,METALS
77,TTKPRESTIG,770.00,93.17,38.0,M-SC,1.53,83218.0,-17559.0,17651.0,-1.35,-17.42,21.21,0.09,245.0,-0.99,0.59,9.33,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.48,47.0,H-MC,2.84,189020.0,24350.0,18978.0,-0.47,14.79,10.04,26.31,89.0,1.28,1.35,23.95,X40N,NTT,AC
34,ICICIGI,2260.25,-17.09,59.0,H-MC,1.78,142613.0,6480.0,24644.0,0.82,4.76,17.28,22.86,91.0,0.26,1.02,17.97,X40,ATH,INSURANCE
57,RELIANCE,1533.00,-14.41,46.0,H-LC,3.40,214344.0,3978.0,24800.0,0.78,1.89,11.57,13.68,37.0,0.16,1.53,18.73,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,RAJOOENG,143.10,-40.28,44.0,H-SC,16.07,90540.0,-11960.0,52558.0,5.00,-11.67,58.05,39.61,114.0,-0.23,0.65,10.58,AR,ATH,MISC
71,TANLA,1963.11,-17.45,59.0,H-SC,18.86,221636.0,-49125.0,379086.0,4.91,-18.14,171.04,121.86,133.0,-0.13,1.58,73.38,AR,ATH,IT
65,SIEMENS,4671.50,1.46,58.0,H-LC,1.94,162965.0,-23130.0,70613.0,3.01,-12.43,43.33,25.51,15.0,-0.33,1.16,20.09,AR,ATH,ELECTRICAL
48,LTIM,7230.20,-4.34,53.0,H-LC,2.00,237195.0,-8956.0,88165.0,2.95,-3.64,37.17,32.18,16.0,-0.10,1.69,31.09,X200,ATH,IT
4,ANGELONE,3033.00,4.59,50.0,H-SC,11.28,185427.0,-5579.0,63286.0,2.73,-2.92,34.13,30.21,157.0,-0.09,1.32,17.35,X40N,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-181.82,62.0,M-SC,11.42,140589.0,-8631.0,152806.0,-5.34,-5.78,108.69,96.62,208.0,-0.06,1.00,52.15,XY25,NTT,FINANCE
66,SIS,528.00,1994.82,30.0,H-SC,3.63,84545.0,-26487.0,50093.0,-3.42,-23.86,59.25,21.26,156.0,-0.53,0.60,14.23,OX40N,NTT,MISC
69,SURYODAY,240.00,64.47,66.0,H-SC,5.93,151286.0,-27785.0,93510.0,-2.42,-15.52,61.81,36.71,135.0,-0.30,1.08,50.11,XR,NTT,BANKS
46,KPIGREEN,731.05,-6.33,33.0,H-SC,7.36,111712.0,-13585.0,72512.0,-2.05,-10.84,64.91,47.03,141.0,-0.19,0.80,39.87,MH,ATH,POWER
53,QUESS,986.00,-44.35,36.0,M-SC,38.25,55188.0,-9818.0,160746.0,-2.00,-15.10,291.27,232.18,198.0,-0.06,0.39,0.00,XY24,NTT,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,ICICIPRULI,790.0,-21.11,48.0,H-MC,2.13,135668.0,-120.0,42871.0,-0.13,-0.09,31.60,31.48,107.0,-0.00,0.97,12.02,X40,ATH,INSURANCE
86,WIPRO,420.0,-14.70,43.0,M-LC,5.66,149891.0,-1054.0,110515.0,0.32,-0.70,73.73,72.51,53.0,-0.01,1.07,5.28,XR,NTT,IT


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,66.0,H-SC,12.83,130977.0,-11502.0,31028.0,0.35,-8.07,23.69,13.70,163.0,-0.37,0.93,54.59,OX40N,NTT,BANKS
45,KANSAINER,340.0,-67.24,58.0,H-SC,2.01,225702.0,-43965.0,80305.0,-0.63,-16.30,35.58,13.47,138.0,-0.55,1.61,14.76,XY24,NTT,PAINTS
77,TTKPRESTIG,770.0,93.17,38.0,M-SC,1.53,83218.0,-17559.0,17651.0,-1.35,-17.42,21.21,0.09,245.0,-0.99,0.59,9.33,OX40N,NTT,DURABLES
18,CERA,9475.0,-22.67,38.0,H-SC,1.47,140921.0,-34982.0,76999.0,-0.35,-19.89,54.64,23.89,149.0,-0.45,1.00,21.70,OX40N,NTT,CERAMICS
72,TATAELXSI,9161.0,-24.38,42.0,H-MC,6.23,102268.0,-29567.0,71792.0,0.50,-22.43,70.20,32.03,98.0,-0.41,0.73,14.55,OX40N,NTT,IT


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1250.88,69.0,M-SC,20.16,183714.0,2571.0,42897.0,-0.43,1.42,23.35,25.10,235.0,0.06,1.31,47.23,XY24,NTT,HEALTHCARE
60,SAIL,228.00,44.35,52.0,M-MC,12.24,228079.0,3117.0,164536.0,-1.86,1.39,72.14,74.53,192.0,0.02,1.62,34.09,XY24,BTT,STEEL
32,HINDZINC,730.22,31.07,68.0,M-LC,5.57,213476.0,8400.0,104176.0,-0.15,4.10,48.80,54.89,52.0,0.08,1.52,29.18,X5K,ATH,METALS
14,BLUESTARCO,2080.00,9.48,47.0,H-MC,2.84,189020.0,24350.0,18978.0,-0.47,14.79,10.04,26.31,89.0,1.28,1.35,23.95,X40N,NTT,AC
84,VOLTAS,1530.00,-2.90,45.0,H-MC,1.63,203775.0,12033.0,25716.0,-0.14,6.28,12.62,19.69,99.0,0.47,1.45,13.50,XY25,NTT,AC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.39,115354.0,14850.0,14258.0,-1.59,14.78,12.36,28.96,79.0,1.04,0.82,53.76,MH,ATH,METALS
11,BANKINDIA,190.00,-25.47,67.0,H-MC,10.15,192250.0,12442.0,99970.0,-0.42,6.92,52.00,62.52,88.0,0.12,1.37,43.37,XR,NTT,BANKS
32,HINDZINC,730.22,31.07,68.0,M-LC,5.57,213476.0,8400.0,104176.0,-0.15,4.10,48.80,54.89,52.0,0.08,1.52,29.18,X5K,ATH,METALS
37,INDIAMART,4810.62,-54.64,37.0,H-SC,9.51,126511.0,3175.0,128447.0,0.66,2.57,101.53,106.72,119.0,0.02,0.90,26.55,AR,ATH,MISC
86,WIPRO,420.00,-14.70,43.0,M-LC,5.66,149891.0,-1054.0,110515.0,0.32,-0.70,73.73,72.51,53.0,-0.01,1.07,5.28,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-21.85,31.0,H-LC,9.77,281198.0,-34644.0,147320.0,-0.61,-10.97,52.39,35.67,5.0,-0.24,2.00,1.37,X40N,ATH,FMCG
5,ASIANPAINT,3465.66,-18.03,33.0,H-LC,5.82,207073.0,-44695.0,97904.0,-0.20,-17.75,47.28,21.13,27.0,-0.46,1.48,11.38,X40,ATH,PAINTS
30,HCLTECH,1937.36,-4.09,36.0,H-LC,10.15,218780.0,-23144.0,85390.0,0.29,-9.57,39.03,25.73,8.0,-0.27,1.56,3.39,X40,ATH,IT
42,ITC,452.00,-39.19,41.0,H-LC,2.04,195771.0,-4367.0,25254.0,-0.98,-2.18,12.90,10.44,4.0,-0.17,1.39,3.26,X40,NTT,FMCG
44,JIOFIN,387.00,-3.82,44.0,H-LC,12.76,261354.0,-6011.0,71454.0,0.73,-2.25,27.34,24.48,48.0,-0.08,1.86,51.49,XY24,BTT,FINANCE


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.5,1.46,58.0,H-LC,1.94,162965.0,-23130.0,70613.0,3.01,-12.43,43.33,25.51,15.0,-0.33,1.16,20.09,AR,ATH,ELECTRICAL
48,LTIM,7230.2,-4.34,53.0,H-LC,2.00,237195.0,-8956.0,88165.0,2.95,-3.64,37.17,32.18,16.0,-0.10,1.69,31.09,X200,ATH,IT
42,ITC,452.0,-39.19,41.0,H-LC,2.04,195771.0,-4367.0,25254.0,-0.98,-2.18,12.90,10.44,4.0,-0.17,1.39,3.26,X40,NTT,FMCG
51,NESTLEIND,1377.0,-10.26,51.0,H-LC,3.23,276190.0,10764.0,46041.0,0.89,4.06,16.67,21.40,11.0,0.23,1.97,10.97,XY25,NTT,FMCG
57,RELIANCE,1533.0,-14.41,46.0,H-LC,3.40,214344.0,3978.0,24800.0,0.78,1.89,11.57,13.68,37.0,0.16,1.53,18.73,XY25,NTT,REFINERIES


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,VBL,671.64,-21.85,31.0,H-LC,9.77,281198.0,-34644.0,147320.0,-0.61,-10.97,52.39,35.67,5.0,-0.24,2.00,1.37,X40N,ATH,FMCG
74,TCS,4406.34,-29.65,44.0,H-LC,12.24,279772.0,-66184.0,134430.0,2.56,-19.13,48.05,19.73,1.0,-0.49,1.99,3.04,X40,ATH,IT
42,ITC,452.00,-39.19,41.0,H-LC,2.04,195771.0,-4367.0,25254.0,-0.98,-2.18,12.90,10.44,4.0,-0.17,1.39,3.26,X40,NTT,FMCG
30,HCLTECH,1937.36,-4.09,36.0,H-LC,10.15,218780.0,-23144.0,85390.0,0.29,-9.57,39.03,25.73,8.0,-0.27,1.56,3.39,X40,ATH,IT
80,UNITDSPR,1644.00,-11.23,57.0,H-LC,6.99,232544.0,-1600.0,50230.0,-0.78,-0.68,21.60,20.77,86.0,-0.03,1.66,6.19,X40N,NTT,BREWERIES


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,1.46,58.0,H-LC,1.94,162965.0,-23130.0,70613.0,3.01,-12.43,43.33,25.51,15.0,-0.33,1.16,20.09,AR,ATH,ELECTRICAL
42,ITC,452.00,-39.19,41.0,H-LC,2.04,195771.0,-4367.0,25254.0,-0.98,-2.18,12.90,10.44,4.0,-0.17,1.39,3.26,X40,NTT,FMCG
19,CIPLA,1795.00,-22.18,44.0,H-LC,4.02,207884.0,3384.0,38022.0,-0.02,1.65,18.29,20.25,10.0,0.09,1.48,10.75,X40N,BTT,PHARMA
5,ASIANPAINT,3465.66,-18.03,33.0,H-LC,5.82,207073.0,-44695.0,97904.0,-0.20,-17.75,47.28,21.13,27.0,-0.46,1.48,11.38,X40,ATH,PAINTS
57,RELIANCE,1533.00,-14.41,46.0,H-LC,3.40,214344.0,3978.0,24800.0,0.78,1.89,11.57,13.68,37.0,0.16,1.53,18.73,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-29.65,44.0,H-LC,12.24,279772.0,-66184.0,134430.0,2.56,-19.13,48.05,19.73,1.0,-0.49,1.99,3.04,X40,ATH,IT
40,INFY,2275.00,-19.60,47.0,H-LC,8.14,313067.0,179.0,171498.0,1.60,0.06,54.78,54.87,3.0,0.00,2.23,6.71,X40,BTT,IT
42,ITC,452.00,-39.19,41.0,H-LC,2.04,195771.0,-4367.0,25254.0,-0.98,-2.18,12.90,10.44,4.0,-0.17,1.39,3.26,X40,NTT,FMCG
83,VBL,671.64,-21.85,31.0,H-LC,9.77,281198.0,-34644.0,147320.0,-0.61,-10.97,52.39,35.67,5.0,-0.24,2.00,1.37,X40N,ATH,FMCG
1,ABB,7934.00,-40.06,48.0,H-LC,7.57,250464.0,-11155.0,130367.0,0.67,-4.26,52.05,45.57,7.0,-0.09,1.78,6.57,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-45.67,46.0,L-SC,32.85,80160.0,-33389.0,73443.0,1.26,-29.40,91.62,35.27,268.0,-0.45,0.57,96.19,XR,NTT,HOTELS
49,MASFIN,398.61,-18.70,47.0,H-SC,11.52,92310.0,-5670.0,27278.0,1.28,-5.79,29.55,22.05,152.0,-0.21,0.66,34.18,XR,ATH,FINANCE
50,NATIONALUM,244.55,-42.00,63.0,H-MC,13.39,115354.0,14850.0,14258.0,-1.59,14.78,12.36,28.96,79.0,1.04,0.82,53.76,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,66.0,H-SC,12.83,130977.0,-11502.0,31028.0,0.35,-8.07,23.69,13.70,163.0,-0.37,0.93,54.59,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-181.82,62.0,M-SC,11.42,140589.0,-8631.0,152806.0,-5.34,-5.78,108.69,96.62,208.0,-0.06,1.00,52.15,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
69,SURYODAY,240.00,64.47,66.0,H-SC,5.93,151286.0,-27785.0,93510.0,-2.42,-15.52,61.81,36.71,135.0,-0.30,1.08,50.11,XR,NTT,BANKS
61,SAMMAANCAP,326.00,-181.82,62.0,M-SC,11.42,140589.0,-8631.0,152806.0,-5.34,-5.78,108.69,96.62,208.0,-0.06,1.00,52.15,XY25,NTT,FINANCE
64,SHALBY,327.00,1250.88,69.0,M-SC,20.16,183714.0,2571.0,42897.0,-0.43,1.42,23.35,25.10,235.0,0.06,1.31,47.23,XY24,NTT,HEALTHCARE
32,HINDZINC,730.22,31.07,68.0,M-LC,5.57,213476.0,8400.0,104176.0,-0.15,4.10,48.80,54.89,52.0,0.08,1.52,29.18,X5K,ATH,METALS
26,GREENPANEL,537.00,225.58,56.0,M-SC,2.92,145424.0,-37654.0,116630.0,-0.35,-20.57,80.20,43.14,230.0,-0.32,1.04,36.79,XY24,NTT,MISC


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.07
1,25,43.26
2,50,74.92


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    44.17
LC    32.12
MC    23.71
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.90
X40      14.55
X40N     11.22
XR       11.02
XY25     10.34
AR        8.26
OX40N     7.73
X200      1.69
MH        1.62
X5K       1.52
SR        1.15
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.07
H-LC    25.59
H-MC    20.48
M-SC    13.67
M-LC     5.53
M-MC     2.93
L-SC     1.43
L-LC     1.00
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.79,-5.92,40.83
IT,12.66,-18.31,82.40
FINANCE,10.42,-15.41,67.83
MISC,6.95,-17.78,83.03
BANKS,6.35,-11.86,70.71
PAINTS,5.57,-18.22,35.84
ELECTRICAL,5.36,-9.26,47.59
INSURANCE,3.77,-4.82,44.00
AUTO,3.54,-15.95,71.37


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3237532.0,22
XR,1363145.0,14
AR,1169887.0,9
X40,798522.0,10
X40N,630394.0,9
OX40N,561318.0,10
XY25,484638.0,7
SR,260890.0,2
X5K,104176.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3712118.0,29
M-SC,1513800.0,17
H-MC,1230634.0,15
H-LC,1219046.0,15
M-LC,373736.0,4
M-MC,371178.0,2
L-SC,261055.0,3
L-MC,60547.0,1
L-LC,43323.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1220035.0      6
M-SC       XY24       848007.0      7
H-SC       AR         800768.0      5
           XR         789736.0      7
H-MC       XY24       567813.0      4
H-LC       X40        514476.0      5
M-MC       XY24       371178.0      2
H-SC       X40N       309172.0      4
           SR         260890.0      2
           OX40N      259005.0      4
H-LC       X40N       235572.0      3
H-MC       X40        211041.0      4
H-LC       AR         200980.0      2
H-MC       XY25       180110.0      2
L-SC       XR         172218.0      2
M-SC       AR         168139.0      2
M-LC       XY24       159045.0      2
M-SC       XY25       152806.0      1
           OX40N      141684.0      4
           XR         130159.0      2
M-LC       XR         110515.0      1
H-LC       XY25       108399.0      3
M-LC       X5K        104176.0      1
H-MC       XR          99970.0      1
L-SC       OX40N       88837.0      1
H-LC       X200        88165.0      1
H-MC       X40N        85650.0      2
M-SC       X40         73005.0      1
H-SC       MH          72512.0      1
H-MC       OX40N       71792.0      1
H-LC       XY24        71454.0      1
L-MC       XR          60547.0      1
L-LC       XY25        43323.0      1
H-MC       MH          14258.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
